In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple
import numpy as np
# from geneticalgorithm import geneticalgorithm as ga
import pandas as pd
from collections import Counter
from tqdm import tqdm
import time
from Bio.SeqUtils import MeltingTemp
from Bio import SeqIO
from plotly import graph_objects as go
import json
from imp import reload
# import primer_selection
# reload(primer_selection)
# import testing
# reload(testing)


/tmp/ipykernel_3663247/859605501.py:18: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [7]:
len('GCTTGCATGCCTGCAGGTCGACTCTAGAGGATCCCCCTACATTTTACACACAGGAAACAGCTATGACCATG')

71

In [14]:
from primer3 import bindings


In [19]:
import random

def generate_random_gene_seq(length):
    bases = ['A', 'T', 'C', 'G']
    return ''.join(random.choice(bases) for _ in range(length))

random_gene_seq = generate_random_gene_seq(300)
# print(f"Random Gene Sequence of Length 100: {random_gene_seq}")


In [23]:
random_gene_seq

'CGTCTGTACATGCCGCTCTATTACAATTATTCCACAAGGGTCAAGGGGCCGCTGCGCTCGGAACGCTCGTGATAGCGCTATCTCAACGCTATTACGGTAGATTCGACTTAAACACATACCTAAGCGCAAGGTAGCACGGCGGGCGTCTTTTAAACTTGGGCATCGCGACCACCGGCCAATTACGGTTTAGCTATCGTCTTACTTAGGCTATAGAGCGCTTTGCCAGAACAAAGGAAAGCGTAAGGATAACCACGGTTCGATGTTGATGGGCGACGACAGGTAGCTCGATCCTCCACTCCG'

In [22]:
import primer3

# Define sequence arguments
seq_args = {
    'SEQUENCE_ID': 'example',
    'SEQUENCE_TEMPLATE': random_gene_seq,
    'SEQUENCE_INCLUDED_REGION': [15, len(random_gene_seq) - 15]
}

# Define global arguments
global_args = {
    'PRIMER_TASK': 'generic',
    'PRIMER_PICK_LEFT_PRIMER': 1,
    'PRIMER_PICK_INTERNAL_OLIGO': 1,
    'PRIMER_PICK_RIGHT_PRIMER': 1,
    'PRIMER_OPT_SIZE': 20,
    'PRIMER_MIN_SIZE': 18,
    'PRIMER_MAX_SIZE': 25,
    'PRIMER_NUM_RETURN': 5,
    'PRIMER_MIN_TM': 57.0,
    'PRIMER_MAX_TM': 63.0,
    'PRIMER_MIN_GC': 20.0,
    'PRIMER_MAX_GC': 80.0,
    'PRIMER_MAX_POLY_X': 100,
    'PRIMER_INTERNAL_MAX_POLY_X': 100,
    'PRIMER_SALT_MONOVALENT': 50.0,
    'PRIMER_DNA_CONC': 50.0,
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_MAX_SELF_ANY': 12,
    'PRIMER_MAX_SELF_END': 8,
    'PRIMER_PAIR_MAX_COMPL_ANY': 12,
    'PRIMER_PAIR_MAX_COMPL_END': 8
}

# Run the primer design
designs = primer3.design_primers(seq_args, global_args)
designs = bindings.design_primers(seq_args, global_args)

# Output results
for key, value in designs.items():
    print(f"{key}: {value}")


PRIMER_LEFT_EXPLAIN: considered 1488, GC content failed 5, low tm 540, high tm 553, high hairpin stability 11, ok 379
PRIMER_RIGHT_EXPLAIN: considered 1488, low tm 458, high tm 490, ok 540
PRIMER_INTERNAL_EXPLAIN: considered 2635, GC content failed 5, low tm 1501, high tm 384, high hairpin stability 26, ok 719
PRIMER_PAIR_EXPLAIN: considered 24, unacceptable product size 18, ok 6
PRIMER_LEFT_NUM_RETURNED: 5
PRIMER_RIGHT_NUM_RETURNED: 5
PRIMER_INTERNAL_NUM_RETURNED: 5
PRIMER_PAIR_NUM_RETURNED: 5
PRIMER_PAIR: [{'PENALTY': 0.42132891917782445, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 119, 'PRODUCT_TM': 83.74517309519567}, {'PENALTY': 0.44012224215788365, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 204, 'PRODUCT_TM': 86.62122351536374}, {'PENALTY': 0.44066222716770653, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 205, 'PRODUCT_TM': 86.7326295411887}, {'PENALTY': 0.4442396702864926, 'COMPL_ANY_TH': 0.0, 'COMPL_END_TH': 0.0, 'PRODUCT_SIZE': 202, 'P

In [12]:
# Print to validate
print("Start of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][0])
print("Length of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][1])
print("Total length of SEQUENCE_TEMPLATE: ", len(seq_args['SEQUENCE_TEMPLATE']))

# Validate bounds
if seq_args['SEQUENCE_INCLUDED_REGION'][0] + seq_args['SEQUENCE_INCLUDED_REGION'][1] > len(seq_args['SEQUENCE_TEMPLATE']):
    print("Error: SEQUENCE_INCLUDED_REGION is out of bounds")


Start of SEQUENCE_INCLUDED_REGION:  1
Length of SEQUENCE_INCLUDED_REGION:  70
Total length of SEQUENCE_TEMPLATE:  71


In [3]:
seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCCACTGCCGATCACGACGATGTTCTTAGCGTCGTAGTCGAGGTCCTCGGGCCAGTGCTGCGGATGGATGATCGGCCCGACGAAATCCTCCGAGCCGGCGAATCTCGGCGAGTAGCCCTCGTCGTAGTTGTAGTAGCCGCTGCACAGAAAGAGGAATTCGCAGGTGAGGGCGCTGAGCGTGCCGTGGCTTTGGATGTGAACGGTCCAGCGGTTTTCCGCGGTCGACCAATCGGCACTGATCACCTTGTGGTGGAACCGGATATGCCTGTCGATTCCATACATGGCCGCGGTGCTCTTGACGTACTCGAGGATGGGCTTGCCGTCGGCGATCGCCTGCCGTCCGGTCCAGGGACGGAATCGGAAACCTAGCGTGTACATGTCGGAGTCGGAGCGAATTCCGGGATAACGGAACAAATCCCAGGTGCCGCCCATGGATTCCCGCTTTTCCAGGATGGCGTAGCTCTTGGTCGGGCAACGGTCCTGCAGGTGCCAGGCCGCGCTGACACCGGAGATTCCAGCGCCCACGATGACAACGTCGAGGTGCTCGGTCATGGATCCACGCTATCAACGTAATGTCGAGGCCGTCAACGAGATGTCGACACTATCGACACGTAGTAAGCTGCCAGGGTGACCACCTCCGCGGCCAGTCAGGCTTCGCTGCCTAGGGGCCGGCGCACCGCGCGGCCGTCCGGCGACGATCGTGAACTGGCGATCCTCGCCACCGCCGAGAACCTTCTCGAGGACCGTCCGCTGGCCGATATCTCGGTCGACGATCTGGCCAAGGGCGCCGGTATCTCGAGGCCGACGTTCTACTTCTATTTCCCATCCAAGGAAGCGGTGCTGCTGACCCTGCTGGACCGGGTGGTCAATCAAGCCGACATGGCCCTACAGACCCTTGCCGAGAATCCCGCCGACACCGACCGCGAGAACATGTGGCGCACCGGGATCAACGTGTTCTTCGAGACATTCGGGTCGCACAAGGCGGTAACCCGAGCCGGTCAGGCCGCCAGGGCAACCAGTGTCGAAGTCGCCGAACTGTGGTCGACGTTTATGCAGAAGTGGATCGCCTACACGGCCGCCGTGATCGACGCCGAACGCGACCGAGGCGCGGCGCCGCGCACCCTGCCGGCCCATGAACTGGCCACAGCGCTCAACCTGATGAACGAGCGGACGCTGTTCGCGTCATTCGCCGGCGAACAGCCCTCGGT'
len(seq)

1500

In [8]:
seq[205:230]

'CACGATGTAGGTGGGTGAGCGCTGC'

In [ ]:
ACTGCCGATCACGACGATGT

In [13]:
left_seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'
len(left_seq)

295

In [10]:
left_seq

'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'

In [12]:
primer_selection.extract_sequence_from_fasta(4327174, 4327174+20, padding=0)

'AACCGGATATGCCTGTCGAT'

In [15]:
primer_selection.extract_sequence_from_fasta(4327129, 4327129+20, padding=0)


'CGGTTTTCCGCGGTCGACCA'

In [14]:
4327129 + 295

4327424

In [36]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence):
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        if query_sequence in sequence:
            start_index = sequence.index(query_sequence)
            end_index = start_index + len(query_sequence)
            return f"Sequence found in {record.id} from position {start_index} to {end_index}"
    return "Sequence not found in the FASTA file"

# Example usage
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "ATCATCAACGGGACCGAGCG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)


Sequence found in Chromosome from position 760286 to 760306


In [40]:
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "AGTCCACCGACAAGACGCTG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)

Sequence found in Chromosome from position 760365 to 760385


In [35]:
4328229-350

4327879

In [41]:
find_sequence_location(fasta_file, 'CCGGTTCAGGCTTCACCACA')

'Sequence found in Chromosome from position 13 to 33'

In [45]:
from Bio import SeqIO

def find_sequence_location(query_sequence, position, fasta_file):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        print("Primer Sequence not found in FASTA file.")
        return 0,0

In [50]:
find_sequence_location('GCCTAAACCGGCGCTAAACC', 4326008, fasta_file='/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa')


(4325973, 4325993)

In [28]:
primer_selection.extract_sequence_from_fasta(4326924, 4326944, padding=0)


'ACTGCCGATCACGACGATGT'

In [25]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence, position):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        return "Sequence not found in the FASTA file"

In [29]:
query_sequence ='ACTGCCGATCACGACGATGT'
# query_sequence = primer_selection.reverse_complement_sequence(query_sequence)
find_sequence_location(fasta_file, query_sequence, 4327932)[0]
# print(result)

4326924

In [ ]:
4327129 4327629

In [ ]:
4327129

In [20]:
[[4326879, 4327879],
 [2282776, 2283776],
 [2285958, 2286958],
 [4246734, 4247734],
 [4244193, 4245193],
 [2280899, 2281899],
 [2152912, 2153912],
 [2284954, 2285954],
 [2747141, 2748141],
 [1834840, 1835840],
 [2150990, 2151990],
 [2277174, 2278174],
 [3067958, 3068958],
 [2147188, 2148188],
 [2138550, 2139550],
 [2281712, 2282712],
 [2283527, 2284527],
 [1416371, 1417371],
 [3840393, 3841393],
 [3073502, 3074502],
 [2149884, 2150884],
 [2276296, 2277296],
 [2279554, 2280554],
 [2287239, 2288239],
 [2141412, 2142412],
 [2140159, 2141159],
 [2148199, 2149199],
 [2144518, 2145518],
 [3086947, 3087947],
 [2128861, 2129861]]

4326629